In [1]:
import torch
import imageio
from PINN import PINN
from Net import Net
import numpy as np
import pandas as pd
import os
import time

In [2]:
data_dir = f"{os.getcwd()}/pinn_test_data"
# load in data
df_wind_ch4 = pd.read_csv(data_dir + "/wind_ch4.csv")
df_true_emission = pd.read_csv(data_dir + "/selected_controll_release.csv")
source_points = np.load(data_dir + "/source_points.npy") # shape=(n_source, 3)
sensor_points = np.load(data_dir + "/sensor_points.npy") # shape=(n_sensor, 3)
#col_points = np.load(data_dir + "/col_points.npy")  # shape=(n_col, 3)
df_bounds = pd.read_csv(data_dir + "/bounds.csv", dtype='float32')
x_min = df_bounds['x_min'][0]
x_max = df_bounds['x_max'][0]
y_min = df_bounds['y_min'][0]
y_max = df_bounds['y_max'][0]
z_min = df_bounds['z_min'][0]
z_max = df_bounds['z_max'][0]

x_max = 1
y_max = 1
z_max = 1
source_location = np.array([[.5,.5,.5]])

ws = df_wind_ch4['wind_speed.m/s'].to_numpy() # shape=(N_t,)
wd = df_wind_ch4['wind_direction'].to_numpy() # shape=(N_t,)
ch4 = np.transpose(df_wind_ch4.iloc[:, 3:].to_numpy()) # shape=(N_obs, N_t)
sensor_names = df_wind_ch4.columns[3:]

In [3]:
model = PINN([50,50,50])
model.set_location(source_location,[1,x_max,y_max,z_max],source_values=[1])
tfinal = 1.

In [4]:
optimizer = torch.optim.Adam(model.net.parameters(), lr=1e-2)
# from torch.optim.lr_scheduler import ExponentialLR

# scheduler = ExponentialLR(optimizer, gamma=0.999)  # Decay LR by 5% every epoch

In [5]:
n= int(5e3)
sn = 250
best_loss = np.inf
sigma=.025
max_epochs = int(5e2)
print_freq = 1

sampling_freq = 10 # how often to resample collocation and source points

for epoch in range(max_epochs):

    start_time = time.time()

    if epoch % sampling_freq == 0:
        # points for source PDE loss. uvpoints are wind field at different times. source_values is the value of source at source_colloc_points (assumes Gaussian for now)
        source_collocation_points,uv_points,source_values = model.source_points(sn,sigma) 
        # initial condition collocation points with smaller time values 0.1*t_final. 
        # ic_col = torch.cat([torch.rand(sn,1)*tfinal*.1, torch.rand(sn,1)*x_max, torch.rand(sn,1)*y_max, torch.rand(sn,1)*z_max], dim=1)
        collocation_points = torch.cat([torch.rand(n,1)*tfinal, torch.rand(n,1)*x_max, torch.rand(n,1)*y_max, torch.rand(n,1)*z_max], dim=1)

        xxx = torch.tensor(model.source_locs).repeat(len(collocation_points),1)
        ten = torch.sqrt(torch.sum(torch.square(collocation_points[:,1:] - xxx),axis=1))
        collocation_points = collocation_points[ten>sigma*3]
    # collocation_points = torch.cat([collocation_points,ic_col])
    # collocation_points.requires_grad=True

    uv = torch.ones(len(collocation_points),2)*.5
    optimizer.zero_grad()

    loss_1 ,pde_1 = model.loss_function(collocation_points,uv) # PDE residual loss
    loss_2,pde_2 = model.loss_function(source_collocation_points, uv_points, source_term = source_values) # source term PDE residual loss 
    loss_3 ,pde_3 = model.loss_function(torch.concat([collocation_points,source_collocation_points]),torch.concat([collocation_points,source_collocation_points]))

    loss = loss_1 + loss_2
    # loss = loss_2*100
    # loss = loss_1

    # # print loss at first epoch
    # if epoch == 0:
    #     print('epoch: %d, loss: %1.3e, pde_res: %1.3e, source_loss: %1.3e' % (epoch, loss.item(), loss_1.item(), loss_2.item()))

    loss.backward()

    # compute norm of gradient of the network
    grad_norm = 0
    for p in model.net.parameters():
        grad_norm += p.grad.data.norm(2).item()**2
    grad_norm = grad_norm**0.5


    if loss.item() < best_loss:
        torch.save(model,'best_mod.m')
    optimizer.step()

    end_time = time.time()
    epoch_time = end_time - start_time
    # scheduler.step()

    if epoch % print_freq == 0:

        # print epoch and loss using %1.3e format
        print('epoch: %d, loss: %1.3e, grad_norm: %1.3e, pde_res: %1.3e, source_loss: %1.3e, time: %1.3e' % (epoch, loss.item(), grad_norm, loss_1.item(), loss_2.item(), epoch_time))

        # print(epoch, loss.item())
        # print(loss_2.item(),loss_1.item())


epoch: 0, loss: 5.616e+01, grad_norm: 3.088e+01, pde_res: 6.264e-02, source_loss: 5.610e+01, time: 1.992e-01
epoch: 1, loss: 5.038e+01, grad_norm: 2.648e+01, pde_res: 9.728e-02, source_loss: 5.028e+01, time: 1.353e-01
epoch: 2, loss: 4.549e+01, grad_norm: 2.906e+01, pde_res: 4.942e-01, source_loss: 4.500e+01, time: 1.677e-01
epoch: 3, loss: 3.973e+01, grad_norm: 3.148e+01, pde_res: 1.570e+00, source_loss: 3.816e+01, time: 1.506e-01
epoch: 4, loss: 3.377e+01, grad_norm: 2.791e+01, pde_res: 4.143e+00, source_loss: 2.962e+01, time: 1.440e-01
epoch: 5, loss: 2.959e+01, grad_norm: 1.168e+01, pde_res: 9.386e+00, source_loss: 2.020e+01, time: 1.427e-01
epoch: 6, loss: 2.985e+01, grad_norm: 1.970e+01, pde_res: 1.765e+01, source_loss: 1.220e+01, time: 1.391e-01
epoch: 7, loss: 3.281e+01, grad_norm: 4.564e+01, pde_res: 2.446e+01, source_loss: 8.347e+00, time: 1.336e-01
epoch: 8, loss: 3.356e+01, grad_norm: 5.111e+01, pde_res: 2.581e+01, source_loss: 7.753e+00, time: 1.432e-01
epoch: 9, loss: 3.2

KeyboardInterrupt: 

In [ ]:
print(x_max)

In [ ]:
model = torch.load('best_mod.m')

In [ ]:
import matplotlib.pyplot as plt

# load the best model 
Z_value = .5
# net.load_state_dict(torch.load('best_model.pth'))

# Define the grid and time steps
n= 100
x_grid = np.linspace(0, x_max, n)
y_grid = np.linspace(0, y_max, n)
z_grid = np.linspace(0, z_max, n)

X, Y, = np.meshgrid(x_grid, y_grid)
Z= X * 0 + Z_value

grid_points = np.vstack([X.ravel(), Y.ravel(), Z.ravel()]).T
# grid_points = np.vstack([X.ravel(), Y.ravel(), Z.ravel()]).T  # Flatten the grid
grid_points = torch.tensor(grid_points, dtype=torch.float32)

time_steps = np.linspace(0, tfinal, 20)  # 10 time steps from 0 to 1


'''
# Plot the concentration over time
for t in time_steps:
    t_tensor = torch.full((grid_points.shape[0], 1), t, dtype=torch.float32)  # Time input
    concentration = net(grid_points, t_tensor).cpu().detach().numpy().reshape(100, 100)  # Predict and reshape
    
    plt.figure()
    plt.contourf(X, Y, concentration, levels=50, cmap='viridis', vmin=0, vmax=10)  # Plot concentration as a contour plot
    plt.plot(source_loc[0,0].cpu(), source_loc[0,1].cpu(), 'ro', label='Source Location')  # Plot the source location
    plt.colorbar(label='Concentration')
    # fix colorbar from 0 to 1 
    # plt.clim(0, 10.0)  # Set colorbar limits
    plt.title(f"Gas Concentration at t = {t:.2f}")
    plt.xlabel('x')
    plt.ylabel('y')
    # fix colorbar 
    # plt.savefig(f"concentration_t_{t:.2f}.png")  # Save the plot as an image
'''

In [ ]:
# save as a GIF
import imageio
import os

images = []
# source_loc = torch.tensor([[.5,.5,.5]])
source_loc = model.source_locs
for t in time_steps:
    t_tensor = torch.full((grid_points.shape[0], 1), t, dtype=torch.float32)  # Time input
    concentration = model.forward(torch.cat([t_tensor,grid_points],dim=1),scaled=True).cpu().detach().numpy().reshape(100, 100)  # Predict and reshape
    
    plt.figure()
    plt.contourf(X, Y, concentration, levels=50, cmap='viridis', vmin=0, vmax=10.0)  # Plot concentration as a contour plot
    plt.plot(model.source_locs[0,0], model.source_locs[0,1], 'ro', label='Source Location')  # Plot the source location
    plt.colorbar(label='Concentration')
    plt.title(f"Gas Concentration at t = {t:.2f}")
    plt.xlabel('x')
    plt.ylabel('y')
    plt.savefig(f"concentration_t_{t:.2f}.png")  # Save the plot as an image
    plt.close()
    
    images.append(imageio.imread(f"concentration_t_{t:.2f}.png"))  # Append the image to the list
    os.remove(f"concentration_t_{t:.2f}.png")  # Remove the image file

imageio.mimsave(f'test_visual.gif', images)  # Save the images as a GIF



In [ ]:
print(len(model.net.hidden))

In [ ]:
print(source_points)

In [ ]:
print(source_values)